In [1]:
import numpy as np

In [2]:
class FourInARow:

    def __init__(self):
        
        self.arraySize = arraySize = 42
        self.width = width = 7

        self.board = np.zeros(arraySize)
        self.status = 'Ongoing'
        self.winner = 0
        
        self.winningCombinations = np.array([
            [0, 1, 2, 3],
            [0, width, 2 * width, 3 * width],
            [0, width + 1, 2 * width + 2, 3 * width + 3],
            [3, width + 2, 2 * width + 1, 3 * width]
        ])
        
    
    def get_action_size(self):
        return self.width

    def get_available_actions(self, board):
        actions = np.zeros(self.width)
        for i in range(self.width):
            if np.any(board[i::self.width]==0):
                actions[i] = 1
        return actions
    
    def get_board_from_player(self, player):
        return self.board * player

    def check_if_winner(self, player):
        
        arraySize = self.arraySize
        width = self.width
        
        for combo in self.winningCombinations:

            for pos in range(arraySize):
                
                #print(combo, pos)
                shifted_combo = combo + pos
                combo_rows = shifted_combo // width

                # check if i'm already outside the board
                if max(combo_rows) >= arraySize // width: continue

                # exclude combinations spanning the wrong numbers of rows
                differenceRows = max(combo_rows) - min(combo_rows)
                if differenceRows != 3 and differenceRows != 0: continue

                #console.log(pos, shifted_combo, differenceRows)

                #console.log(shifted_combo.every((el) => game[el] === player))
                if (np.array([self.board[el] for el in shifted_combo]) == player).all():
                    #print(player, " wins with ", shifted_combo)
                    self.status = "Winner"
                    self.winner = player
                    return True
        
        return False
    
    def play(self, board):
        self.board = board.copy()
           
    def get_reward_for_player(self, board, player):
        arraySize = self.arraySize
        width = self.width
        
        for combo in self.winningCombinations:

            for pos in range(arraySize):
                
                shifted_combo = combo + pos
                combo_rows = shifted_combo // width

                # check if i'm already outside the board
                if max(combo_rows) >= arraySize // width: continue

                # exclude combinations spanning the wrong numbers of rows
                differenceRows = max(combo_rows) - min(combo_rows)
                if differenceRows != 3 and differenceRows != 0: continue

                if (np.array([board[el] for el in shifted_combo]) == player).all():
                    self.status = "Winner"
                    self.winner = player
                    return 1.0
                elif (np.array([board[el] for el in shifted_combo]) == -1*player).all():
                    self.status = "Winner"
                    self.winner = -1*player
                    return -1.0
                
        if np.any(board==0):
            return None
        
        self.status = "Draw"
        return 0
    
    def next_player(self):
        board = list(self.board)
        if board.count(1) == board.count(-1):
            self.player = 1
        else:
            self.player = -1
        return self.player
    
    def next_state(self, board, player, action):
        self.board = board
        for pos in range(35+action,-1,-self.width):
            if self.board[pos]==0:
                self.board[pos] = player
                break
        player = self.next_player()
        return self.board, player

coordinates

In [3]:
np.arange(42).reshape(6,7)

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27],
       [28, 29, 30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39, 40, 41]])

In [4]:
fiar = FourInARow()

assert the action size

In [5]:
assert fiar.get_action_size()==7

assert the available actions

In [6]:
assert (fiar.get_available_actions(np.zeros(42))==np.ones(7)).all()
assert (fiar.get_reward_for_player(np.zeros(42),1)==None)
assert (fiar.get_reward_for_player(np.zeros(42),-1)==None)
fiar.play(np.zeros(42))
assert (fiar.check_if_winner(1)==False )
assert (fiar.check_if_winner(-1)==False )



assert (fiar.get_available_actions(np.ones(42))==np.zeros(7)).all()
assert (fiar.get_reward_for_player(np.ones(42),1)==1)
assert (fiar.get_reward_for_player(np.ones(42),-1)==-1)
fiar.play(-1*np.ones(42))
assert (fiar.check_if_winner(1)==False )
assert (fiar.check_if_winner(-1)==True )

In [7]:
case = np.zeros(42)
case[-7:] = 1

columns_to_fill = [0,3]
for c in columns_to_fill:
    case[c::7] = 1


solution = np.ones(7)
solution[columns_to_fill] = 0

print(case.reshape(6,7))

[[1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1.]]


In [8]:
assert (fiar.get_available_actions(case)==solution).all()

fiar.play(case)

assert (fiar.check_if_winner(1)==True )
assert (fiar.check_if_winner(-1)==False )
assert (fiar.get_reward_for_player(np.ones(42),1)==1)
assert (fiar.get_reward_for_player(np.ones(42),-1)==-1)

assert (fiar.get_board_from_player(player=1)==case).all()
assert (fiar.get_board_from_player(player=-1)==-case).all()

fiar.play(case*-1)

assert (fiar.check_if_winner(1)==False )
assert (fiar.check_if_winner(-1)==True )
assert (fiar.get_reward_for_player(-np.ones(42),1)==-1)
assert (fiar.get_reward_for_player(-np.ones(42),-1)==1)

assert (fiar.get_board_from_player(player=1)==-case).all()
assert (fiar.get_board_from_player(player=-1)==case).all()

In [9]:
next_player = 1
next_state, next_player = fiar.next_state(board=np.zeros(42), player=next_player, action=0)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=0)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=3)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=2)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=5)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=6)
print(next_state.reshape(6,7))

print("Last board for player 1")
print(fiar.get_board_from_player(player=1).reshape(6,7))
print("Last board for player -1")
print(fiar.get_board_from_player(player=-1).reshape(6,7))

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0. -1.  1.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0. -1.  1.  0.  1.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 

# Test TF model

In [10]:
import numpy as np
import tensorflow as tf

In [11]:
inputs = fiar.get_board_from_player(player=-1)[np.newaxis,:]

In [12]:
x = tf.cast(tf.one_hot(tf.cast(inputs + 1, tf.int32), 3), tf.float32)
x = tf.reshape(x, (-1, 6, 7, 3))
x = tf.keras.layers.Conv2D(
        filters=256, kernel_size=(4, 4)
    )(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Conv2D(
        filters=256, kernel_size=(2,3)
    )(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
flatten_inputs = tf.keras.layers.Flatten()(inputs)
x = tf.concat([flatten_inputs, x], axis=1)
residual_tower = tf.keras.layers.ReLU(name="residual_tower")(x)
print(residual_tower)

tf.Tensor([[0.         0.         0.         ... 0.00659088 0.         0.        ]], shape=(1, 1066), dtype=float32)


In [13]:
def create_az_model():

    kernel_regularizer = tf.keras.regularizers.L2(1e-4)

    inputs = tf.keras.layers.Input((42))

    # One-Hot encoding
    x = tf.cast(tf.one_hot(tf.cast(inputs + 1, tf.int32), 3), tf.float32)

    # Residual tower
    x = tf.reshape(x, (-1, 6, 7, 3))
    x = tf.keras.layers.Conv2D(
        filters=256, kernel_size=(4, 4), kernel_regularizer=kernel_regularizer
    )(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv2D(
        filters=256, kernel_size=(2,3), kernel_regularizer=kernel_regularizer
    )(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Flatten()(x)
    flatten_inputs = tf.keras.layers.Flatten()(inputs)
    x = tf.concat([flatten_inputs, x], axis=1)
    residual_tower = tf.keras.layers.ReLU(name="residual_tower")(x)

    # Policy
    x = tf.keras.layers.Conv1D(
        filters=2, kernel_size=1, kernel_regularizer=kernel_regularizer
    )(residual_tower[..., np.newaxis])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Flatten()(x)
    p = tf.keras.layers.Dense(
        7,
        activation=tf.keras.activations.softmax,
        kernel_regularizer=kernel_regularizer,
        name="policy",
    )(x)

    # Value
    x = tf.keras.layers.Conv1D(
        filters=1, kernel_size=1, kernel_regularizer=kernel_regularizer
    )(residual_tower[..., np.newaxis])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(
        256,
        activation=tf.keras.activations.relu,
        kernel_regularizer=kernel_regularizer,
    )(x)
    v = tf.keras.layers.Dense(
        1, activation=tf.nn.tanh, kernel_regularizer=kernel_regularizer, name="value"
    )(x)

    outputs = p, v

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="fiar_model")

    # zero_weights = []
    # for arr in model.get_weights():
    #     zero_weights.append(np.zeros(arr.shape))
    # model.set_weights(zero_weights)

    return model


In [14]:
model = create_az_model()

In [15]:
model(inputs)

(<tf.Tensor: shape=(1, 7), dtype=float32, numpy=
 array([[0.14285715, 0.14285715, 0.14285715, 0.14285715, 0.14285715,
         0.14285715, 0.14285715]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.]], dtype=float32)>)

# MCTS

In [16]:
from libs.MCTS import MCTS

In [17]:
mcts = MCTS(game=FourInARow(), n_simulations=100)

In [18]:
state = np.zeros(42)

In [19]:
root = mcts.run(model=model, state=state, player=1)

In [20]:
root

State:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Player:1
Value:0.0
Leaf:False
Visits:100
Children:
 -0: prior=0.14285714285714285
 -1: prior=0.14285714285714285
 -2: prior=0.14285714285714285
 -3: prior=0.14285714285714285
 -4: prior=0.14285714285714285
 -5: prior=0.14285714285714285
 -6: prior=0.14285714285714285

In [21]:
root.children[2]

State:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
Player:-1
Value:0.0
Leaf:False
Visits:14
Children:
 -0: prior=0.14285714285714285
 -1: prior=0.14285714285714285
 -2: prior=0.14285714285714285
 -3: prior=0.14285714285714285
 -4: prior=0.14285714285714285
 -5: prior=0.14285714285714285
 -6: prior=0.14285714285714285

In [22]:
root.children[2].children

{0: State:
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.
   0.  1.  0.  0.  0.  0.]
 Player:1
 Value:0.0
 Leaf:False
 Visits:2
 Children:
  -0: prior=0.14285714285714285
  -1: prior=0.14285714285714285
  -2: prior=0.14285714285714285
  -3: prior=0.14285714285714285
  -4: prior=0.14285714285714285
  -5: prior=0.14285714285714285
  -6: prior=0.14285714285714285,
 1: State:
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  -1.  1.  0.  0.  0.  0.]
 Player:1
 Value:0.0
 Leaf:False
 Visits:2
 Children:
  -0: prior=0.14285714285714285
  -1: prior=0.14285714285714285
  -2: prior=0.14285714285714285
  -3: prior=0.14285714285714285
  -4: prior=0.14285714285714285
  -5: prior=0.14285714285714285
  -6: prior=0.14285714285714285,
 2: State:
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

# Training

In [31]:
from libs.Trainer import Trainer
import time
import multiprocessing as mp

n_cores = mp.cpu_count()
print(f"[Number of cores]: {n_cores}")

[Number of cores]: 4


In [35]:
start_time = time.time()
trainer = Trainer(game = FourInARow, mcts = MCTS, model=create_az_model())
dataset = trainer.create_dataset(number_of_games=200, temperature=1)
print("Running time: %s seconds" % (time.time() - start_time))

Running time: 7412.072238206863 seconds


In [36]:
import pickle
with open("200_tictactoe.pickle", "wb") as fp:
    pickle.dump(dataset, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
def parallel_fn(n):
    trainer = Trainer(game = FourInARow, mcts = MCTS, model=create_az_model())
    game_batch = trainer.create_dataset(number_of_games=1, temperature=1)
    return game_batch

In [34]:
start_time = time.time()
with mp.Pool(n_cores) as pool:
    dataset = pool.map(parallel_fn, range(4))
print("Running time: %s seconds" % (time.time() - start_time))

KeyboardInterrupt: 

In [26]:
dataset

[(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0.14, 0.14, 0.16, 0.14, 0.14, 0.14, 0.14]),
  -1.0),
 (array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
         -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
         -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
         -1., -0., -0.]),
  array([0.14285714, 0.14285714, 0.14285714, 0.14285714, 0.14285714,
         0.14285714, 0.14285714]),
  1.0),
 (array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,
          1.,  0.,  0.]),
  array([0.14035088, 0.15789474, 0.15789474, 0.14035088, 0.12280702,
         0.15789474, 0.12280702]),
  -1.0),
 (array([-0., -0., -